In [8]:
import json
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

# ----------- Step 1: Load Dataset -----------
def parse_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

df = parse_data('Sarcasm_Headlines_Dataset_v2.json')

# ----------- Step 2: Split by Sarcasm Label -----------
df_sarcastic = df[df['is_sarcastic'] == 1].reset_index(drop=True)
df_non_sarcastic = df[df['is_sarcastic'] == 0].reset_index(drop=True)

print(f"Sarcastic headlines: {len(df_sarcastic)}")
print(f"Non-sarcastic headlines: {len(df_non_sarcastic)}")

# ----------- Step 3: Load SentenceTransformer Model -----------
model = SentenceTransformer('all-mpnet-base-v2')

# ----------- Step 4: Encode Headlines -----------
sarcastic_embeddings = model.encode(df_sarcastic['headline'].tolist(), convert_to_tensor=True, show_progress_bar=True)
non_sarcastic_embeddings = model.encode(df_non_sarcastic['headline'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# ----------- Step 5: Semantic Search to Create Pairs -----------
print("Finding best sarcastic match for each non-sarcastic headline...")

hits = util.semantic_search(non_sarcastic_embeddings, sarcastic_embeddings, top_k=1)
pseudo_pairs = []

for i, hit in enumerate(hits):
    target_idx = hit[0]['corpus_id']
    pseudo_pairs.append({
        'input': df_non_sarcastic.iloc[i]['headline'],
        'target': df_sarcastic.iloc[target_idx]['headline']
    })

df_pairs = pd.DataFrame(pseudo_pairs)
df_pairs['input'] = 'sarcasmify: ' + df_pairs['input']

# ----------- Step 6: Save or Display Sample -----------
df_pairs.to_csv("pseudo_sarcasm_pairs.csv", index=False)
print("\n✅ Sample pairs:")
print(df_pairs.head())


Sarcastic headlines: 13634
Non-sarcastic headlines: 14985


Batches: 100%|██████████| 469/469 [00:16<00:00, 28.49it/s]


Finding best sarcastic match for each non-sarcastic headline...

✅ Sample pairs:
                                               input  \
0  sarcasmify: dem rep. totally nails why congres...   
1  sarcasmify: eat your veggies: 9 deliciously di...   
2                   sarcasmify: my white inheritance   
3  sarcasmify: 5 ways to file your taxes with les...   
4     sarcasmify: lots of parents know this scenario   

                                              target  
0  presumptuous congressional freshman thinks she...  
1    vegetarian option just iceberg lettuce on bread  
2  report: all the other races coming to take you...  
3  woman going to take quick break after filling ...  
4  new babysitter can already tell this kind of k...  


In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_pairs).train_test_split(test_size=0.1)

In [57]:

# ======================================
# ⚙️ TOKENIZER & TRAINING FUNCTION
# ======================================
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

def train_model(model_name, model_class, tokenizer_class, dataset, epoch = 3, dataset_name = "ori"):
    model = model_class.from_pretrained(model_name)
    tokenizer = tokenizer_class.from_pretrained(model_name)

    def preprocess(examples):
        inputs = tokenizer(examples["input"], truncation=True, padding="max_length", max_length=64)
        targets = tokenizer(examples["target"], truncation=True, padding="max_length", max_length=64)
        inputs["labels"] = targets["input_ids"]
        return inputs

    tokenized = dataset.map(preprocess, batched=True)
    args = TrainingArguments(
        output_dir=f"./{model_name.replace('/', '_')}_{dataset_name}_sarcasm",
        evaluation_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=epoch,
        learning_rate=2e-4,
        weight_decay=0.01,
        save_total_limit=2,
        logging_dir="./logs"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    )
    trainer.train()
    return model, tokenizer

# ======================================
# 🧠 TRAIN MODELS
# ======================================
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    BartForConditionalGeneration, BartTokenizer
)

In [58]:
t5_model, t5_tokenizer = train_model("t5-small", T5ForConditionalGeneration, T5Tokenizer, dataset)

Map: 100%|██████████| 1499/1499 [00:00<00:00, 11827.52 examples/s]
/Users/samuelthen/Documents/CS4248/cs4248-project/.venv/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/zr/tc7r4hpj6tscx9pf08qck6m80000gn/T/ipykernel_27705/1849639684.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [59]:
bart_model, bart_tokenizer = train_model("facebook/bart-base", BartForConditionalGeneration, BartTokenizer, dataset)

Map: 100%|██████████| 1499/1499 [00:00<00:00, 5885.26 examples/s]
/Users/samuelthen/Documents/CS4248/cs4248-project/.venv/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/zr/tc7r4hpj6tscx9pf08qck6m80000gn/T/ipykernel_27705/1849639684.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/Users/samuelthen/Documents/CS4248/cs4248-project/.venv/lib/python3.12/site-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [60]:
flan_model, flan_tokenizer = train_model("google/flan-t5-small", T5ForConditionalGeneration, T5Tokenizer, dataset)

Map: 100%|██████████| 1499/1499 [00:00<00:00, 11980.69 examples/s]
/Users/samuelthen/Documents/CS4248/cs4248-project/.venv/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/zr/tc7r4hpj6tscx9pf08qck6m80000gn/T/ipykernel_27705/1849639684.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# ======================================
# 🧠 GPT-2 MODEL (Separate Preprocessing)
# ======================================
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling

gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

def preprocess_gpt2(examples):
    prompts = [inp + ". " + tgt for inp, tgt in zip(examples["input"], examples["target"])]
    model_inputs = gpt2_tokenizer(prompts, truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = model_inputs["input_ids"]
    return model_inputs

gpt2_tokenized = dataset.map(preprocess_gpt2, batched=True, remove_columns=["input", "target"])
gpt2_collator = DataCollatorForLanguageModeling(tokenizer=gpt2_tokenizer, mlm=False)
gpt2_args = TrainingArguments(
    output_dir="./gpt2_sarcasm",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    logging_steps=10,
)

gpt2_trainer = Trainer(
    model=gpt2_model,
    args=gpt2_args,
    data_collator=gpt2_collator,
    train_dataset=gpt2_tokenized["train"],
)
gpt2_trainer.train()

In [67]:
# ======================================
# 🎭 TEST ON SAMPLE HEADLINES
# ======================================
test_headlines = [
    "jaguar land rover pauses shipments to u.s. amid new tariffs",
    "president trump's tariffs trigger global market turmoil",
    "global reaction to trump's sweeping tariff measures",
    "potential worldwide recession feared due to u.s. tariffs",
    "vietnam, india, and israel in talks over new trade deals",
    "u.s. supreme court allows freeze on dei-focused teacher training grants",
    "chevron ordered to pay $744 million for environmental damages",
    "surge in measles cases reported in texas",
    "los angeles county settles $4 billion abuse case in juvenile facilities",
    "anti-trump protests erupt across the u.s.",
    "marsiling-yew tee to receive new community spaces and parks",
    "political parties debate balance of foreign talent in singapore",
    "opel to cease vehicle sales in singapore by end-2025",
    "singapore's economy projected to grow by 2.6% in 2025",
    "temporary bus diversions for singapore t100 2025 event"
]


def generate(model, tokenizer, headline, is_gpt2=False):
    if is_gpt2:
        model.to("cpu")
        prompt = f"sarcasmify: {headline}."
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        output = model.generate(input_ids, max_length=64, do_sample=True, top_k=50, top_p=0.95, temperature=0.9, pad_token_id=tokenizer.eos_token_id)
    else:
        model.to("cpu")
        prompt = "sarcasmify: " + headline
        input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True)
        output = model.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ======================================
# 📊 COMPARE MODELS SIDE-BY-SIDE
# ======================================
from IPython.display import display, Markdown

for h in test_headlines:
    t5_out = generate(t5_model, t5_tokenizer, h)
    bart_out = generate(bart_model, bart_tokenizer, h)
    flan_out = generate(flan_model, flan_tokenizer, h)
    # gpt2_out = generate(gpt2_model, gpt2_tokenizer, h, is_gpt2=True)

    display(Markdown(f"""
### 📰 Original: `{h}`  
| Model | Output |
|-------|--------|
| T5        | `{t5_out}` |
| Bart      | `{bart_out}` |
| FLAN-T5   | `{flan_out}` |
"""))


### 📰 Original: `jaguar land rover pauses shipments to u.s. amid new tariffs`  
| Model | Output |
|-------|--------|
| T5        | `jaguar land rover announces plans to expand jaguar land rover` |
| Bart      | `lone tent a dark harbinger of looming street festival` |
| FLAN-T5   | `jaguar land rover announces plans to re-establish jaguar land rover` |



### 📰 Original: `president trump's tariffs trigger global market turmoil`  
| Model | Output |
|-------|--------|
| T5        | `trump warns u.s. not to compromise on u.s. tariffs` |
| Bart      | `world wonders what trump has on united states that's forcing nation to keep him in power` |
| FLAN-T5   | `trump unveils plan to address climate change by reviving climate change` |



### 📰 Original: `global reaction to trump's sweeping tariff measures`  
| Model | Output |
|-------|--------|
| T5        | `trump warns u.s. not to tolerate u.s. tariff hikes` |
| Bart      | `trump warns china not to underestimate his willingness to sacrifice every american's well-being` |
| FLAN-T5   | `trump unveils plan to address climate change by reviving climate change` |



### 📰 Original: `potential worldwide recession feared due to u.s. tariffs`  
| Model | Output |
|-------|--------|
| T5        | `report: u.s. has no idea what to expect from u.s.` |
| Bart      | `report: it not good time for long, devastating war for iran, either` |
| FLAN-T5   | `u.s. warns americans not to underestimate u.s. economic woes` |



### 📰 Original: `vietnam, india, and israel in talks over new trade deals`  
| Model | Output |
|-------|--------|
| T5        | `israel, vietnam, india, iran, israel, u.s., u.s., u.s., u.s., u.s., u.s., u.s., u.s.` |
| Bart      | `israeli government found to be in league with jewry` |
| FLAN-T5   | `israel, u.s., u.s., u.s., u.s., u.s., u.s., u.s., u.s.` |


...


### 📰 Original: `marsiling-yew tee to receive new community spaces and parks`  
| Model | Output |
|-------|--------|
| T5        | `tee taylor swift announces plans to reopen city hall` |
| Bart      | `new park caters to business travelers` |
| FLAN-T5   | `taylor swift announces plans to build new taylor swift stadium` |



### 📰 Original: `political parties debate balance of foreign talent in singapore`  
| Model | Output |
|-------|--------|
| T5        | `fbi: 'it's a good time to think about it'` |
| Bart      | `republicans, democrats unite in good laugh over reform party` |
| FLAN-T5   | `nation's foreign policy experts recommend breaking down crushing defeats into smaller, more manageable failures` |



### 📰 Original: `opel to cease vehicle sales in singapore by end-2025`  
| Model | Output |
|-------|--------|
| T5        | `opel announces plan to stop sales of cars in singapore` |
| Bart      | `report: there never been a better time to buy than right now` |
| FLAN-T5   | `opel announces plan to reduce emissions by 30% by 2025` |



### 📰 Original: `singapore's economy projected to grow by 2.6% in 2025`  
| Model | Output |
|-------|--------|
| T5        | `report: singapore expected to grow by 2.6% in 2025` |
| Bart      | `report: middle east quickly running out of land area for violence to spill over to` |
| FLAN-T5   | `report: u.s. economy projected to grow by 2025` |



### 📰 Original: `temporary bus diversions for singapore t100 2025 event`  
| Model | Output |
|-------|--------|
| T5        | `singapore t100 reopens for first time since 2001` |
| Bart      | `bus transporting carnival cruise passengers crashes into sewage treatment plant` |
| FLAN-T5   | `tuesday tuesday tuesday tuesday` |


In [38]:
len(pseudo_pairs)

14985

In [40]:
import openai
import time
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()
client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# ----------- Step 4: Generate New Sarcastic Headlines Using OpenAI -----------
generated_data = []

def create_prompt(pair):
    return (
        f"Example sarcastic headline (style guide only): \"{pair['target']}\"\n"
        f"Non-sarcastic headline to convert: \"{pair['input']}\"\n\n"
        "Write a sarcastic version of the non-sarcastic headline. "
        "Do NOT copy or refer to the content of the example. "
        "Only mimic its tone, structure, and sarcasm style.\n"
        "Only return the sarcastic headline. No explanation or intro."
    )

# Progress bar over the whole list
pbar = tqdm(total=len(pseudo_pairs), desc="Generating sarcasm")

for idx, pair in enumerate(pseudo_pairs):
    prompt = create_prompt(pair)

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an expert sarcasm writer."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=60,
            temperature=0.8
        )
        output = response.choices[0].message.content.strip()

        generated_data.append({
            "input": "sarcasmify: " + pair["input"],
            "reference": pair["target"],
            "generated": output
        })
        pbar.update(1)

    except Exception as e:
        print(f"❌ Error at index {idx}: {e}")
        continue

    time.sleep(1)  # Respect rate limits

pbar.close()

# ----------- Step 5: Save Results -----------
df_results = pd.DataFrame(generated_data)
df_results.to_csv("gpt_sarcasm_augmented_dataset.csv", index=False)
print("\n✅ Saved enhanced sarcastic dataset with GPT-generated headlines:")
print(df_results.head())


Generating sarcasm: 100%|██████████| 14985/14985 [12:36:22<00:00,  3.03s/it]  


✅ Saved enhanced sarcastic dataset with GPT-generated headlines:
                                               input  \
0  sarcasmify: dem rep. totally nails why congres...   
1  sarcasmify: eat your veggies: 9 deliciously di...   
2                   sarcasmify: my white inheritance   
3  sarcasmify: 5 ways to file your taxes with les...   
4     sarcasmify: lots of parents know this scenario   

                                           reference  \
0  presumptuous congressional freshman thinks she...   
1    vegetarian option just iceberg lettuce on bread   
2  report: all the other races coming to take you...   
3  woman going to take quick break after filling ...   
4  new babysitter can already tell this kind of k...   

                                           generated  
0  "brilliant congresswoman suggests that actuall...  
1  "feast on your greens: 9 ways to make broccoli...  
2  "Study: Everyone's just dying to get their han...  
3  "5 Ingenious Techniques to Turn Your 

In [41]:
print(df_results.head().to_string(index=False))


                                                                                      input                                                                                  reference                                                                                                                   generated
sarcasmify: dem rep. totally nails why congress is falling short on gender, racial equality presumptuous congressional freshman thinks she can just come in and represent constituents "brilliant congresswoman suggests that actually addressing gender and racial equality might be a good idea—groundbreaking!"
                              sarcasmify: eat your veggies: 9 deliciously different recipes                                            vegetarian option just iceberg lettuce on bread                                                      "feast on your greens: 9 ways to make broccoli a culinary masterpiece"
                                                           sarcasmify: my white

In [42]:
df_results.rename(columns={"generated": "target"}, inplace=True)

In [61]:
dataset_enhanced = Dataset.from_pandas(df_results).train_test_split(test_size=0.1)

In [69]:
t5_model_enhanced, t5_tokenizer_enhanced = train_model("t5-base", T5ForConditionalGeneration, T5Tokenizer, dataset_enhanced, dataset_name="enhanced")

Map: 100%|██████████| 1499/1499 [00:00<00:00, 11089.31 examples/s]
/Users/samuelthen/Documents/CS4248/cs4248-project/.venv/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/zr/tc7r4hpj6tscx9pf08qck6m80000gn/T/ipykernel_27705/1849639684.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
bart_model_enhanced, bart_tokenizer_enhanced = train_model("facebook/bart-base", BartForConditionalGeneration, BartTokenizer, dataset_enhanced, dataset_name="enhanced")

Map: 100%|██████████| 1499/1499 [00:00<00:00, 5274.33 examples/s]
/Users/samuelthen/Documents/CS4248/cs4248-project/.venv/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/zr/tc7r4hpj6tscx9pf08qck6m80000gn/T/ipykernel_27705/1849639684.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/Users/samuelthen/Documents/CS4248/cs4248-project/.venv/lib/python3.12/site-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [70]:
flan_model_enhanced, flan_tokenizer_enhanced = train_model("google/flan-t5-base", T5ForConditionalGeneration, T5Tokenizer, dataset_enhanced, dataset_name="enhanced")

Map: 100%|██████████| 1499/1499 [00:00<00:00, 10496.53 examples/s]
/Users/samuelthen/Documents/CS4248/cs4248-project/.venv/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/zr/tc7r4hpj6tscx9pf08qck6m80000gn/T/ipykernel_27705/1849639684.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [71]:
for h in test_headlines:
    t5_out_enhanced = generate(t5_model_enhanced, t5_tokenizer_enhanced, h)
    bart_out_enhanced = generate(bart_model_enhanced, bart_tokenizer_enhanced, h)
    flan_out_enhanced = generate(flan_model_enhanced, flan_tokenizer_enhanced, h)

    display(Markdown(f"""
### 📰 Original: `{h}`  
| Model | Output |
|-------|--------|
| T5        | `{t5_out_enhanced}` |
| Bart      | `{bart_out_enhanced}` |
| FLAN-T5   | `{flan_out_enhanced}` |
"""))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



### 📰 Original: `jaguar land rover pauses shipments to u.s. amid new tariffs`  
| Model | Output |
|-------|--------|
| T5        | `"Jaguar land rover bravely decides to stop shipments to the U.S., because who doesn’t love a little chaos?"` |
| Bart      | `"Jaguar Land rover pauses shipments to the U.S. because who doesn’t love a good trade war?"` |
| FLAN-T5   | `"Jaguar land rover decides to stop shipments to the U.S., because who doesn’t love a little tariff drama?"` |



### 📰 Original: `president trump's tariffs trigger global market turmoil`  
| Model | Output |
|-------|--------|
| T5        | `"President Trump's tariffs: because who doesn't love a little chaos in the global economy?"` |
| Bart      | `"President Trump’s tariffs spark global chaos—because who doesn’t love a good chaos?"` |
| FLAN-T5   | `"President Trump's tariffs: because who doesn't love a good trade war?"` |



### 📰 Original: `global reaction to trump's sweeping tariff measures`  
| Model | Output |
|-------|--------|
| T5        | `"global reaction to trump's sweeping tariff measures: because who doesn't love a little chaos in their lives?"` |
| Bart      | `"global reaction to trump's latest tariffs: because who doesn’t love a little chaos?"` |
| FLAN-T5   | `"global reaction to trump's groundbreaking tariff measures: because who doesn't love a good surprise?"` |



### 📰 Original: `potential worldwide recession feared due to u.s. tariffs`  
| Model | Output |
|-------|--------|
| T5        | `"Global economy poised for a thrilling rollercoaster ride thanks to U.S. tariffs"` |
| Bart      | `"Experts predict global recession will be just a minor inconvenience thanks to U.S. tariffs"` |
| FLAN-T5   | `"U.S. tariffs: because who doesn’t love a good economic meltdown?"` |


...


### 📰 Original: `marsiling-yew tee to receive new community spaces and parks`  
| Model | Output |
|-------|--------|
| T5        | `"Marsiling-Yew Tee: Because Who Needs Community Spaces When You Have Parks?"` |
| Bart      | `" marsiling-yew tee to revolutionize community spaces and parks, because clearly that’s what the world needs right now"` |
| FLAN-T5   | `"marsiling-yew tee thrilled to receive new community spaces and parks, because why not?"` |



### 📰 Original: `political parties debate balance of foreign talent in singapore`  
| Model | Output |
|-------|--------|
| T5        | `"Political parties grapple with the delicate balance of foreign talent in Singapore—because who doesn’t love a good debate?"` |
| Bart      | `"Political parties expertly navigate the thrilling world of foreign talent insingapore"` |
| FLAN-T5   | `"political parties debate if foreign talent is just a trendy accessory in singapore"` |



### 📰 Original: `opel to cease vehicle sales in singapore by end-2025`  
| Model | Output |
|-------|--------|
| T5        | `"opel announces groundbreaking plan to stop selling cars in singapore by the end of 2025"` |
| Bart      | `"OPL announces grand plans to cease vehicle sales insingapore by end-2025, because who needs customers anyway?"` |
| FLAN-T5   | `"Opel announces groundbreaking plan to stop selling vehicles in Singapore by the end of 2025—because who doesn’t love a good nostalgia trip?"` |



### 📰 Original: `singapore's economy projected to grow by 2.6% in 2025`  
| Model | Output |
|-------|--------|
| T5        | `"Singapore's economy poised to grow by 2.6% in 2025—because who doesn't love a good economic forecast?"` |
| Bart      | `"singapore's economy set to magically grow by 2.6% in 2025—because who needs growth anyway?"` |
| FLAN-T5   | `"Singapore's economy predicted to grow by a staggering 2.6% in 2025—because who doesn't love a good surprise?"` |



### 📰 Original: `temporary bus diversions for singapore t100 2025 event`  
| Model | Output |
|-------|--------|
| T5        | `"Singapore T100 2025: Because Who Doesn't Love a Good Bus Diversion?"` |
| Bart      | `"singapore t100 2025 event: because who doesn’t want to relive the glory days of overpriced bus diversions?"` |
| FLAN-T5   | `"Singapore T100 2025: Because Who Doesn't Love a Good Bus Diversion?"` |
